In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from urllib.parse import urljoin
import time

driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.zomato.com/bangalore")
time.sleep(2)

scroll_pause = 2
screen_height = driver.execute_script("return window.screen.height;")
i = 1
V = 0
with open('Table1.csv', 'a+', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    header = ['Name', 'Price_For_One', 'Cuisines', 'Rating', 'Urls']
    writer.writerow(header)


while True:
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    i = i + 1
    time.sleep(scroll_pause)
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if i == 200:
        break

src = driver.page_source
soup = BeautifulSoup(src, "html.parser")
all_data = soup.find_all("div", class_="sc-1mo3ldo-0")
rest = all_data[6:]

for parent in rest:
    child = parent.find_all("div", class_="jumbo-tracker")
    for child2 in child:
        # get the link
        link_tag = child2.find("a")
        if 'href' in link_tag.attrs:
            link = link_tag.get('href')
            base = "https://www.zomato.com"
            link_new = urljoin(base, link)
            print(link_new)
        else:
            link_new = ''

        # get the rating
        rating_tag = child2.find("div", class_="sc-1q7bklc-1 cILgox")
        if rating_tag is not None:
            rating = rating_tag.text
            print(rating)
        else:
            rating = ''

        # get the name
        name_tag = child2.find("h4")
        if name_tag is not None:
            name = name_tag.text
            print(name)
        else:
            name = ''



        # get the cusine
        cusine_tag = child2.find_all("p", class_ ="sc-1hez2tp-0")
        cusine = ''
        for i in range(len(cusine_tag)):
            if 'for one' not in cusine_tag[i].text and i==3:
                Cusine = cusine_tag[i]
                cusine = Cusine.text
                print(cusine)
            elif "for one" in cusine_tag[i].text and i==3:
                Cusine = cusine_tag[i-1]
                cusine = Cusine.text
                print(cusine)


        # get the price 
        price_tags = child2.find_all("p", class_="sc-1hez2tp-0")
        for price_tag in price_tags:
            if "for one" in price_tag.text:
                one_person_price = price_tag.text
                print(one_person_price)
            

                
            
                        

        # write to csv
        row = [name, one_person_price, cusine, rating, link_new]
        with open('Table1.csv', 'a+', encoding='UTF8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(row)

        

    driver.quit()
    